In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from awpy import Demo
import os
import uuid
from cs_bahtml.radars import RADARS
from cs_bahtml.plotting import position_transform, add_player

In [3]:
dem = Demo(file="..\\..\\demos\\astralis-vs-faze-m1-nuke.dem")

In [17]:
dem.ticks.query("tick == 11378").current_equip_value

113780    4400
113781    4600
113782    4000
113783    3850
113784    5600
113785    5500
113786    5500
113787    4600
113788    4600
113789    4200
Name: current_equip_value, dtype: uint32

In [42]:
(11783-11378)/64

6.328125

In [30]:
dem.grenades.query("thrower == 'device' and entity_id == '121'")

,thrower_steamid,thrower,grenade_type,tick,X,Y,Z,entity_id,round
8239,76561197987713664,device,incendiary_grenade,11783,1643.15625,-1226.09375,-329.28125,121,3
8241,76561197987713664,device,incendiary_grenade,11784,1630.75,-1232.6875,-324.375,121,3
8243,76561197987713664,device,incendiary_grenade,11785,1618.3125,-1239.28125,-319.53125,121,3
8245,76561197987713664,device,incendiary_grenade,11786,1605.90625,-1245.875,-314.8125,121,3
8247,76561197987713664,device,incendiary_grenade,11787,1593.46875,-1252.46875,-310.125,121,3
...,...,...,...,...,...,...,...,...,...
8580,76561197987713664,device,incendiary_grenade,11907,103.1875,-2043.75,-317.1875,121,3
8586,76561197987713664,device,incendiary_grenade,11908,90.78125,-2050.34375,-321.96875,121,3
8592,76561197987713664,device,incendiary_grenade,11909,78.34375,-2056.9375,-326.84375,121,3
8598,76561197987713664,device,incendiary_grenade,11910,65.9375,-2063.53125,-331.78125,121,3


In [31]:
11911-11783

128

In [4]:
map = "de_ancient"
radar = RADARS[map].copy()
radar.show()


In [9]:
radar = RADARS[map].copy()
# Create a draw object
# Define the coordinates of the dot
x, y = -808.5625, -664.84375
player_rad = position_transform(map, 60, "x") - position_transform(map, 0, "x")
radar = add_player(radar, position_transform(map,x,"x"), position_transform(map,y,"y"), player_rad, "t")
# Show the image with the dot
radar.show()

In [31]:
def parse_demo(demo_object,maximum_roundtime= 30,rotate_seconds = 10):
    map_name = demo_object.header.map_name
    tick_rate = 64
    grenades_df = demo_object.grenades
    interesting_grenades = ["smoke","molotov","incendiary_grenade"]
    grenades_df = grenades_df[grenades_df.grenade_type.isin(interesting_grenades)]
    grenades_df.loc[grenades_df['grenade_type'] == 'incendiary_grenade', 'grenade_type'] = 'molotov'
    tick_df = demo_object.ticks
    team_info = tick_df.groupby(["round","side"]).first()['clan'].reset_index()
    round_df = demo_object.rounds
    buy_df = tick_df.merge(round_df['freeze_end'], left_on='tick', right_on='freeze_end', how='left')
    buy_df = buy_df.groupby(['round','side'])['current_equip_value'].mean().reset_index()
    #TODO: Maybe also look at money to gauge whether its a force?
    buy_df['buy_type'] = buy_df.current_equip_value.apply(lambda x: "eco" if x < 1000 else "force" if x < 3000 else "full")

    out_df = grenades_df.groupby(['entity_id',"round"]).first().reset_index()
    out_df['map_name'] = map_name
    out_df = out_df.merge(round_df[["round","freeze_end"]], on='round', how='left')
    out_df['throwSeconds'] = (out_df["tick"] - out_df["freeze_end"])/tick_rate
    out_df = out_df.query("throwSeconds < @maximum_roundtime") 
    
    last_tick_grenade = grenades_df.groupby(['entity_id','round']).last().reset_index()
    out_df = out_df.merge(last_tick_grenade, on=['entity_id','round'], how='left', suffixes=('', '_last_tick'))
    out_df = out_df.merge(tick_df, left_on=['tick','thrower'], right_on=['tick', "name"], how='left', suffixes=('', '_thrower'))
    out_df['throwCords'] = list(zip(out_df['X_thrower'], out_df['Y_thrower'], out_df['Z_thrower']))
    out_df['landCords'] = list(zip(out_df['X_last_tick'], out_df['Y_last_tick'], out_df['Z_last_tick']))
    out_df['throwerSide'] = out_df['side']
    out_df['opponentSide'] = out_df['throwerSide'].apply(lambda x: "TERRORIST" if x == "CT" else "CT")
    
    out_df = out_df.merge(buy_df, on=['round','side'], how='left')
    out_df = out_df.merge(buy_df, left_on=['round','opponentSide'],right_on=['round','side'], how='left', suffixes = ("","_opponentbuy"))
    out_df['throwerBuyType'] = out_df['buy_type']
    out_df['opponentBuyType'] = out_df['buy_type_opponentbuy']
    

    out_df = out_df.merge(team_info, left_on = ["round","opponentSide"], right_on = ["round", "side"], suffixes = ("","_opponent"))
    out_df['throwerTeam'] = out_df['clan']
    out_df['opponentTeam'] = out_df['clan_opponent']
    out_df['throwerName'] = out_df['thrower']
    out_df['rotate_tick'] = out_df['tick']+rotate_seconds*tick_rate

    player_df = out_df.merge(tick_df, left_on=['tick', 'throwerSide'], right_on=['tick', 'side'], how='left', suffixes=('', '_players_thrown'))
    player_df = player_df.query("health>0")
    player_df['position_players_thrown'] = list(zip(player_df['X_players_thrown'], player_df['Y_players_thrown'], player_df['Z_players_thrown']))
    starting_positions = player_df.groupby(['entity_id', 'round'])['position_players_thrown'].agg(list).reset_index(name="starting_positions")
    player_df = player_df.merge(tick_df, left_on=['rotate_tick', 'throwerSide', 'name_players_thrown'], right_on=['tick', 'side', 'name'], how='left', suffixes=('', '_players'))
    player_df['position_players'] = list(zip(player_df['X_players'], player_df['Y_players'], player_df['Z_players']))
    dead_player_df = player_df[player_df['health_players'] == 0].groupby(['entity_id', 'round'])['position_players'].agg(list).reset_index(name='dead_players')
    alive_players_df = player_df[player_df['health_players'] > 0].groupby(['entity_id', 'round'])['position_players'].agg(list).reset_index(name='alive_players')
    out_df = out_df.merge(dead_player_df, on=['entity_id', 'round'], how='left')
    out_df = out_df.merge(alive_players_df, on=['entity_id', 'round'], how='left')
    out_df = out_df.merge(starting_positions, on=['entity_id', 'round'], how='left')
    out_df.dropna(subset=['alive_players'], inplace=True)
    selected_columns = ['map_name', 'entity_id', 'round', 'throwSeconds','grenade_type', 'throwerName', 'throwerTeam', 'throwerBuyType','opponentTeam', 'opponentBuyType', 'throwerSide', 'throwCords', 'landCords', "starting_positions" ,'dead_players', 'alive_players']
    return out_df[selected_columns]

In [32]:
test = parse_demo(dem)
test

,map_name,entity_id,round,throwSeconds,grenade_type,throwerName,throwerTeam,throwerBuyType,opponentTeam,opponentBuyType,throwerSide,throwCords,landCords,starting_positions,dead_players,alive_players
0,de_nuke,65,10,8.71875,smoke,br0,Astralis,full,FaZe Clan,full,CT,"(840.4488525390625, 70.94966125488281, -415.96...","(112.40625, -326.4375, -413.96875)","[(946.9803466796875, -512.65771484375, -96.746...",NaN,"[(675.0065307617188, -654.7633056640625, -206...."
1,de_nuke,65,18,28.328125,smoke,device,Astralis,full,FaZe Clan,full,TERRORIST,"(-1351.2012939453125, -1043.841796875, -370.96...","(1138.6875, -2040.9375, -413.96875)","[(51.53950881958008, -1027.559814453125, -415....",NaN,"[(107.97000122070312, -1016.2108154296875, -41..."
2,de_nuke,66,15,19.140625,molotov,rain,FaZe Clan,full,Astralis,full,CT,"(1167.05517578125, -1296.9703369140625, -415.9...","(1230.15625, -2312.9375, -410.65625)","[(601.146240234375, -984.1541137695312, -415.9...","[(104.07331848144531, 136.4063262939453, -415....","[(596.6456298828125, -1100.789794921875, -365...."
3,de_nuke,71,6,23.515625,molotov,stavn,Astralis,full,FaZe Clan,full,CT,"(861.3890991210938, -1634.232177734375, -415.4...","(1210.71875, -2321.375, -413.375)","[(308.9505615234375, -561.36865234375, -127.96...","[(923.7300415039062, -1557.551025390625, -415....","[(316.55828857421875, -362.82666015625, -127.9..."
4,de_nuke,92,17,19.421875,molotov,device,Astralis,full,FaZe Clan,force,TERRORIST,"(417.908935546875, -1339.753173828125, -415.96...","(879.21875, -1664.59375, -412.96875)","[(544.7725830078125, -818.166015625, -399.9687...","[(619.3079223632812, -812.4690551757812, -415....","[(667.1747436523438, -768.351806640625, -415.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,de_nuke,470,8,5.96875,smoke,rain,FaZe Clan,full,Astralis,full,TERRORIST,"(-640.515380859375, -1418.1844482421875, -415....","(442.25, -2129.75, -413.96875)","[(-892.8883666992188, -1128.3846435546875, -41...","[(219.63461303710938, -2124.313720703125, -415...","[(320.7699890136719, -2034.265380859375, -415...."
158,de_nuke,474,10,27.3125,smoke,karrigan,FaZe Clan,full,Astralis,full,TERRORIST,"(-125.35107421875, -1183.811767578125, -110.79...","(963.9375, -446.6875, -125.96875)","[(-125.35107421875, -1183.811767578125, -110.7...",NaN,"[(-370.3466796875, -1632.046142578125, -415.96..."
159,de_nuke,474,21,8.609375,molotov,ropz,FaZe Clan,full,Astralis,force,CT,"(922.6604614257812, 41.81367111206055, -415.96...","(106.46875, -124.5625, -413.46875)","[(919.3548583984375, -1674.396240234375, -415....",NaN,"[(522.0472412109375, -1249.5416259765625, -415..."
160,de_nuke,475,18,6.921875,smoke,device,Astralis,full,FaZe Clan,full,TERRORIST,"(-1351.1507568359375, -1043.4605712890625, -37...","(1136.90625, -2043.34375, -413.96875)","[(-539.0313720703125, -1147.4931640625, -186.5...",NaN,"[(-311.94775390625, -701.6260986328125, -415.9..."


In [59]:
test.query("throwerName == 'device' and entity_id == '121'")

,map_name,entity_id,round,throwSeconds,grenade_type,throwerName,throwerTeam,opponentTeam,throwerSide,throwCords,landCords,starting_positions,dead_players,alive_players
15,de_nuke,121,3,6.328125,molotov,device,Astralis,FaZe Clan,CT,"(1668.0035400390625, -1212.8868408203125, -401...","(53.5, -2070.125, -336.78125)","[(1146.7567138671875, -444.86102294921875, -15...",NaN,"[(455.9765319824219, -891.97998046875, -251.46..."


In [55]:
test['throwSeconds'] = (test["tick"] - test["freeze_end"])/64

In [51]:
(11783-11378)/64

6.328125

In [33]:
import pandas as pd
data_dir = "..\\..\\demos"
files = os.listdir(data_dir)
data_list = []
for file in files:
    path = os.path.join(data_dir,file)
    print(path)
    try:
        dem = Demo(file=path)
        df = parse_demo(dem,maximum_roundtime= 30,rotate_seconds = 15)
        data_list.append(df)
    except:
        True
df = pd.concat(data_list).reset_index(drop=True)

..\..\demos\astralis-vs-faze-m1-ancient.dem


..\..\demos\astralis-vs-faze-m1-nuke.dem
..\..\demos\astralis-vs-faze-m2-ancient.dem
..\..\demos\astralis-vs-faze-m2-nuke.dem
..\..\demos\astralis-vs-faze-m3-inferno.dem


2024-05-03 06:14:58.803 | WARNING  | awpy.parsers:parse_bomb:501 - bomb_defused not found in events.


..\..\demos\pov
..\..\demos\steel-helmet-vs-astralis-ancient-p1.dem


2024-05-03 06:15:17.871 | WARNING  | awpy.parsers:parse_bomb:471 - bomb_planted not found in events.
2024-05-03 06:15:17.871 | WARNING  | awpy.parsers:parse_bomb:501 - bomb_defused not found in events.
2024-05-03 06:15:17.871 | WARNING  | awpy.parsers:parse_bomb:531 - bomb_exploded not found in events.


..\..\demos\steel-helmet-vs-astralis-ancient-p2.dem
..\..\demos\steel-helmet-vs-astralis-ancient-p3.dem


2024-05-03 06:15:27.903 | WARNING  | awpy.parsers:parse_bomb:501 - bomb_defused not found in events.


..\..\demos\virtus-pro-vs-astralis-m1-overpass-p1.dem
..\..\demos\virtus-pro-vs-astralis-m1-overpass-p2.dem
..\..\demos\virtus-pro-vs-astralis-m2-vertigo.dem


2024-05-03 06:15:50.366 | WARNING  | awpy.parsers:parse_bomb:501 - bomb_defused not found in events.


In [34]:
df 

,map_name,entity_id,round,throwSeconds,grenade_type,throwerName,throwerTeam,throwerBuyType,opponentTeam,opponentBuyType,throwerSide,throwCords,landCords,starting_positions,dead_players,alive_players
0,de_ancient,65,7,19.984375,molotov,jabbi,Astralis,force,FaZe Clan,full,TERRORIST,"(-1545.947021484375, 590.7413330078125, 73.03125)","(-1448.5625, 118.78125, 94.15625)","[(-950.135986328125, -755.3543701171875, 91.39...","[(-1266.0162353515625, 670.6183471679688, 59.5...","[(-1944.0313720703125, 465.9612731933594, 75.0..."
1,de_ancient,65,11,0.515625,smoke,br0,Astralis,full,FaZe Clan,full,TERRORIST,"(-392.0, -2224.0, -117.941162109375)","(-808.5625, -664.84375, 41.84375)","[(-409.4080810546875, -2295.257568359375, -163...",NaN,"[(-1341.12841796875, -1152.1937255859375, -12...."
2,de_ancient,65,12,8.578125,smoke,Staehr,Astralis,full,FaZe Clan,full,TERRORIST,"(-1182.977783203125, -936.0553588867188, 34.62...","(-1134.09375, -870.0, 32.65625)","[(-1182.977783203125, -936.0553588867188, 34.6...",NaN,"[(-775.75439453125, -879.1306762695312, 38.709..."
3,de_ancient,71,7,8.046875,molotov,Staehr,Astralis,force,FaZe Clan,full,TERRORIST,"(-1135.5604248046875, -880.2910766601562, 34.5...","(-382.5, -524.5625, 365.0)","[(-1135.5604248046875, -880.2910766601562, 34....","[(-1266.0162353515625, 670.6183471679688, 59.5...","[(-1437.0113525390625, -1099.208740234375, -6...."
4,de_ancient,74,11,11.484375,molotov,karrigan,FaZe Clan,full,Astralis,full,CT,"(1201.6058349609375, 606.605224609375, 125.605...","(1152.25, -728.75, 15.375)","[(-253.7420654296875, -192.224365234375, 68.03...",NaN,"[(51.93663787841797, -679.7330322265625, 158.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,de_vertigo,1012,2,8.09375,smoke,Staehr,Astralis,full,Virtus.Pro,full,TERRORIST,"(-906.1011962890625, -121.74517822265625, 1148...","(-681.90625, -942.71875, 11630.21875)","[(-1273.4239501953125, -661.0948486328125, 117...",NaN,"[(-1793.464111328125, -534.680419921875, 11552..."
1093,de_vertigo,1014,2,9.34375,molotov,Staehr,Astralis,full,Virtus.Pro,full,TERRORIST,"(-814.6753540039062, -333.866943359375, 11488....","(-1076.15625, -1122.3125, 11778.1875)","[(-1423.9686279296875, -824.96875, 11776.03125...",NaN,"[(-1897.8319091796875, -421.1993408203125, 115..."
1094,de_vertigo,1016,16,12.59375,molotov,Jame,Virtus.Pro,full,Astralis,full,TERRORIST,"(-1423.96875, -824.96875, 11776.03125)","(-1057.75, -723.875, 12018.03125)","[(-818.6741333007812, -507.5389404296875, 1152...","[(-2416.347412109375, -81.03173828125, 11712.0...","[(-913.0172119140625, -549.021484375, 11592.03..."
1095,de_vertigo,1017,8,28.015625,smoke,device,Astralis,full,Virtus.Pro,full,TERRORIST,"(-1807.3677978515625, 545.7968139648438, 11776...","(-1523.59375, 778.78125, 11778.03125)","[(-1807.3677978515625, 545.7968139648438, 1177...","[(-692.0317993164062, -473.999755859375, 11514...","[(-1394.15625, 753.0313110351562, 11776.03125)..."


In [35]:
df.to_json('../data/parsed_util.json')